## NSR ASR NOTEBOOK
This notebook walkthrough the steps required to determine NSRs location in Hong Kong. 

### Prerequisites: 
<input type="checkbox" disabled checked> Python 3.X
<br>
<input type="checkbox" disabled checked> Arcpy
<br>
<input type="checkbox" disabled checked>requests
<br>
<input type="checkbox" disabled checked>Pandas

### Methodology:

#### Steps:
1. Download iG1000 from LandsD HKMS 2.0 from https://www.hkmapservice.gov.hk/OneStopSystem/map-search
2. Download GeoCom csv from LandsD HKMS 2.0
3. Import the required library (Run Code 1)

    ```python
    import requests
    import arcpy 
    import pandas as pd

    ```
4. Extract NSR from GeoCom

| NSR Type | GeoCom Class | GeoCom Type | Type Description |
|:---:|:---:|:---:|---|
|Court of law|GOV|JUD|Magistracy / District Court|
|Domestic premises|--|--|--|
|Educational institution|SCH|CCC<br>KDG<br>MIS<br>PRS<br>SCM<br>SEC<br>SES<br>TEI<br>VTI|Child Care Centre<br>Kindergarten<br>Riding School / Road Safety Centre<br>Primary School<br>Educational Branch / Sub-campus<br>Special Education School<br>Secondary School<br>Post-Secondary<br>Vocational Training Institute / Career Skill Development|
|Hospital|HNC|HOS|Hospital|
|Hostel|AMD|GHS<br>HCV<br>YHL|Guest House<br>Holiday Camp / Village<br>Youth Hostel|
|Hotel|AMD|HTL|Hotel|
|Library|CUF|LIB|Library|
|Medical clinic|HNC<br><br><br>GOV|CLI<br>ELD<br>REH<br>CSD|Clinic / Health Centre / Dispensary<br>Home for the Elderly<br>Rehabilitation Centre<br>Drug Abuse Treatment and Rehabilitation Services|
|Performing arts centre|CUF|PFM|Performing Arts Venue|
|Place of public worship|REM|CHU<br>MON<br>MOS<br>RPS<br>SMY<br>SYN<br>TMP|Church<br>Monastery / Nunnery<br>Mosque<br>Other Religious Meeting Place<br>Seminary<br>Synagogue<br>Temple|
|Temporary housing accommodation|--|--|--|




In [1]:
## Code 1
import requests
import arcpy 
import pandas as pd
import os

In [16]:

GeoCom = r"GeoCom4.0_202112.csv"
arcpy.env.workspace = os.getcwd()

GeoCom_df = pd.read_csv(GeoCom,index_col=False,usecols=['ENGLISHNAME','CHINESENAME','CLASS','TYPE','EASTING','NORTHING','E_ADDRESS','C_ADDRESS'])

GeoCom_df = GeoCom_df.query('TYPE in ["JUD","CCC","KDG","MIS","PRS","SCM","SEC","SES","TEI","VTI","HOS","GHS","HCV","YHL","HTL","LIB","CLI","ELD","REH","CSD","PFM","CHU","MON","MOS","RPS","SMY","SYN","TMP"]')

GeoCom_df['Sources']= 'GeoCom'
GeoCom_csv = GeoCom_df.to_csv("GeoCom_NSR.csv",encoding='utf-8-sig')

GDB = r'NSR_Generation.gdb'
arcpy.env.overwriteOutput = True


arcpy.management.XYTableToPoint("GeoCom_NSR.csv","GeoCom_Point","EASTING","NORTHING",None,arcpy.SpatialReference(2326))


FileNotFoundError: [Errno 2] No such file or directory: './GeoCom_NSR.csv'

ExecuteError: ERROR 000582: Error occurred during execution.


In [ ]:
def addNSR(URL);

    response = requests.get(URL)
    Temp_list = []
    
    #Check status
    if response.status_code == 200:
        json = response.json()
        
        for event in json:
            temp = event[['Name of Block(s)']['en']].split('<br>')
            temp = list(filter(None, temp))
            for item in temp:
                Temp_list.append(item)
                
    else:
        print("Failed",response)
    
    
    
    return

In [2]:
##PRH
response = requests.get(r"https://www.housingauthority.gov.hk/datagovhk/prh-estates.json")

In [44]:
#HOS
response = requests.get(r"https://www.housingauthority.gov.hk/datagovhk/hos-courts.json")

In [3]:
if response.status_code == 200:
    print(response)
    json = response.json()
else:
    print("Failed",response)

<Response [200]>


In [46]:
HOS=[]

for event in json:
    
    temp = event['Name of Block(s)']['en'].split('<br>')
    temp = list(filter(None, temp))
    for item in temp:
        HOS.append(item)

In [6]:
PRH=[]

for event in json:
    
    temp = event['Name of Block(s)']['en'].split('<br>')
    temp = list(filter(None, temp))
    for item in temp:
        PRH.append(item)

In [7]:
#Number of Public Housing Estate = 234
#Number of Public Housing Estate blocks = 1586 
len(PRH)

1586

In [47]:
#Number of Home Ownership Scheme = 220
len(HOS)

743

In [48]:
##Tak Cheung House
fc = r"C:\Users\Jason\Desktop\ANewR\EPD\CNP\CNP.gdb\Building"
fields = ['ENGLISHBUILDINGNAME','NSR']

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    
    for row in cursor:
        if row[0] in HOS    :
            row[1] = "Y"
            cursor.updateRow(row) 